In [ ]:
# File: fitting.ipynb
# Author: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
import os
import shutil
import random
from datetime import datetime

from frontend import App

app = App.create()

codim_ipc_root = "/tmp/Codim-IPC"
checkout_list = [
    #"Projects/FEMShell/input/multilayer",
    "Projects/FEMShell/input/dress_knife",
    "Projects/FEMShell/input/Rumba_Dancing",
]
app.extra.sparse_clone(
    "https://github.com/ipc-sim/Codim-IPC", codim_ipc_root, checkout_list
)

stage_path = os.path.join(codim_ipc_root, checkout_list[0], "stage.obj")
body_path = os.path.join(codim_ipc_root, checkout_list[1], "shell0.obj")

V, F, S = app.extra.load_CIPC_stitch_mesh(stage_path)
app.asset.add.tri("dress", V, F)
app.asset.add.stitch("glue", S)

V, F = app.mesh.load_tri(body_path)
app.asset.add.tri("body", V, F)

scene = app.scene.create("fit-and-dance")
scene.add("dress").stitch("glue").rotate(-90, "x")

jitter = 0.01 * random.random()
body = scene.add("body").at(0, -0.78 + jitter, 0).pin()
body_dir = os.path.join(codim_ipc_root, checkout_list[1])
frame = 1
while True:
    path = os.path.join(body_dir, f"shell{frame}.obj")
    if os.path.exists(path):
        V, _ = app.mesh.load_tri(path)
        body.move_to(V, 0.5 + 0.025 * frame)
        frame += 1
    else:
        break

fixed = scene.build().report()
fixed.preview(options={"pin": False})

In [ ]:
param = (
    app.session.param()
    .set("fitting")
    .set("dt", 0.02)
    .set("frames", 240)
    .set("friction", 0.005)
    .set("model-shell", "baraffwitkin")
    .set("contact-ghat", 0.6e-3)  # 0.5e3  parameter controls the activation threshold of contact. Lower values increase contact density:
    .set("strain-limit-eps", 0.06)   #0.1 Make it smaller to strongly limit how much the cloth can stretch
    .set("cg-tol", 1e-4)
    .set("cg-max-iter",500000)
   .set("bend",20)
    .set("area-density", 8000)
    .set("stitch-stiffness",2.0)
    # .set("target-toi",0.35)  #0.25 is default
    # .set("area-young-mod", 500.0)
    # .set("area-poiss-rat", 0.3)
     # .set("gravity", -11.98)
)
param.dyn("fitting").time(0.25).hold().change(False)
param.dyn("dt").time(0.25).hold().change(0.04) 



# param = (
#     app.session.param()
#     .set("fitting")
#     .set("dt", 0.02)
#     .set("frames", 240)
#     .set("friction", 0.0)
#     .set("model-shell", "arap")
#     .set("contact-ghat", 0.5e-3)
#     .set("strain-limit-eps", 0.1)
# )

session = app.session.create(fixed)
session.start(param).preview()
session.stream()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(position=(0.0, 0.0, 1.8741250038146973), quaterni…

Button(description='Terminate Solver', style=ButtonStyle())

HTML(value='')

HTML(value='')

In [ ]:
def build_scene():
    app = App.create()

    codim_ipc_root = "/tmp/Codim-IPC"
    checkout_list = [
        #"Projects/FEMShell/input/multilayer",
        "Projects/FEMShell/input/dress_knife",
        "Projects/FEMShell/input/Rumba_Dancing",
    ]
    app.extra.sparse_clone(
        "https://github.com/ipc-sim/Codim-IPC", codim_ipc_root, checkout_list
    )

    stage_path = os.path.join(codim_ipc_root, checkout_list[0], "stage.obj")
    body_path = os.path.join(codim_ipc_root, checkout_list[1], "shell0.obj")

    V, F, S = app.extra.load_CIPC_stitch_mesh(stage_path)
    app.asset.add.tri("dress", V, F)
    app.asset.add.stitch("glue", S)

    V, F = app.mesh.load_tri(body_path)
    app.asset.add.tri("body", V, F)

    scene = app.scene.create("fit-and-dance")
    scene.add("dress").stitch("glue").rotate(-90, "x")

    jitter = 0.01 * random.random()
    body = scene.add("body").at(0, -0.78 + jitter, 0).pin()
    body_dir = os.path.join(codim_ipc_root, checkout_list[1])
    frame = 1
    while True:
        path = os.path.join(body_dir, f"shell{frame}.obj")
        if os.path.exists(path):
            V, _ = app.mesh.load_tri(path)
            body.move_to(V, 0.5 + 0.025 * frame)
            frame += 1
        else:
            break

    fixed = scene.build().report()
    fixed.preview(options={"pin": False})

def run_experiment(params, param_dict, exp_name=None):
    build_scene()

    session = app.session.create(fixed)
    session.start(params).preview()
    session.stream()

    while not session.finished():
        pass
    if session.finished():
        print("session finished")

    # Auto-name everything
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    exp_name = exp_name or f"exp_{timestamp}"
    exp_dir = f"./logs/{exp_name}"
    os.makedirs(exp_dir, exist_ok=True)

    # Save stdout separately FIRST (always exists)
    lines_to_write = session.get.log.stdout()
    with open(f"{exp_dir}/stdout.txt", "w") as f:
        f.writelines(line + '\n' for line in lines_to_write)

    # Try to save structured logs
    try:
        logs = session.get.log.names()
        if logs:
            for logname in logs:
                numbers = session.get.log.numbers(logname)
                with open(f"{exp_dir}/{logname}.csv", "w") as f:
                    f.write("frame,value\n")
                    for frame, value in numbers:
                        f.write(f"{frame},{value}\n")
        else:
            print(f"⚠️  Warning: No structured logs found for {exp_name}. PCG might have failed early.")
    except Exception as e:
        print(f"⚠️  Warning: Failed to save structured logs for {exp_name}. Error: {e}")

    # Always save params
    with open(f"{exp_dir}/params.txt", "w") as f:
        for key, value in param_dict.items():
            f.write(f"{key}: {value}\n")

    print(f"✅ Experiment {exp_name} saved to {exp_dir}")

    return session.finished()



In [ ]:
experiments = [
    {"dt": 0.002, "dyn_dt_change": 0.02, "bend": 20, "strain-limit-eps": 0.06,"contact-ghat": 0.4e-3},
    {"dt": 0.002, "dyn_dt_change": 0.02, "bend": 20, "strain-limit-eps": 0.02, "contact-ghat": 0.4e-3},
    {"dt": 0.002, "dyn_dt_change": 0.023, "bend": 20, "strain-limit-eps": 0.01, "contact-ghat": 0.4e-3},    #default strain
    {"dt": 0.002, "dyn_dt_change": 0.023, "bend": 35, "strain-limit-eps": 0.01, "contact-ghat": 0.4e-3},    #default strain
    {"dt": 0.002, "dyn_dt_change": 0.023, "bend": 35, "strain-limit-eps": 0.05, "contact-ghat": 0.4e-3},
    {"dt": 0.002, "dyn_dt_change": 0.023, "bend": 35, "strain-limit-eps": 0.05, "contact-ghat": 0.2e-3},
]

for idx, exp in enumerate(experiments):
    param = (
        app.session.param()
        .set("fitting")
        .set("dt", exp["dt"])
        .set("frames", 240)
        .set("friction", 0.0)
        .set("model-shell", "arap")
        .set("contact-ghat", exp["contact-ghat"])
        .set("strain-limit-eps", exp["strain-limit-eps"])
        .set("cg-max-iter",100000)
        .set("cg-tol",0.00001)
        .set("bend", exp["bend"])
    )
    param.dyn("fitting").time(0.15).hold().change(False)
    param.dyn("dt").time(0.15).hold().change(exp["dyn_dt_change"])

    sess = run_experiment(param, exp, exp_name=f"strain_run_{idx+1}")
    print(sess)




# Define experiment configurations
base_experiments = [
    {"name": "strain_sweep_0.01", "dt": 0.002, "dyn_dt_change": 0.02, "bend": 5, "strain-limit-eps": 0.01, "contact-ghat": 0.4e-3},
    {"name": "strain_sweep_0.05", "dt": 0.002, "dyn_dt_change": 0.02, "bend": 5, "strain-limit-eps": 0.05, "contact-ghat": 0.4e-3},
    {"name": "strain_sweep_0.10", "dt": 0.002, "dyn_dt_change": 0.02, "bend": 2, "strain-limit-eps": 0.1,  "contact-ghat": 0.4e-3},
    {"name": "friction_high",     "dt": 0.005, "dyn_dt_change": 0.02, "bend": 5, "strain-limit-eps": 0.02, "contact-ghat": 0.8e-3, "friction": 0.01},
    {"name": "dt_large",          "dt": 0.05,  "dyn_dt_change": 0.05, "bend": 2, "strain-limit-eps": 0.01, "contact-ghat": 1e-4},
    {"name": "contact_sweep",     "dt": 0.002, "dyn_dt_change": 0.02, "bend": 2, "strain-limit-eps": 0.02, "contact-ghat": 1e-4},
]

shell_models = ["arap", "baraffwitkin"]

# Run all experiments
for exp in base_experiments:
    for shell in shell_models:
        params = (
            app.session.param()
            .set("fitting")
            .set("dt", exp["dt"])
            .set("frames", 240)
            .set("friction", 0.0)
            .set("model-shell", shell)
            .set("contact-ghat", exp["contact-ghat"])
            .set("strain-limit-eps", exp["strain-limit-eps"])
            .set("cg-tol",0.00001)
            .set("cg-max-iter",100000)
            .set("bend", exp["bend"]))
        
        params.dyn("fitting").time(0.15).hold().change(False)
        params.dyn("dt").time(0.15).hold().change(exp["dyn_dt_change"])
        sess= run_experiment(params, exp_name=f"{exp['name']}_{shell}")
        print(sess)
        
        
           # .set("dt", base["dt"]),
           #  "dyn_dt_change": base["dyn_dt_change"],
           #  "bend": base["bend"],
           #  "strain-limit-eps": base["strain-limit-eps"],
           #  "contact-ghat": base["contact-ghat"],
           #  "model-shell": shell,
           #  "frames": 240,
           #  "cg-max-iter": 100000,
           #  "cg-tol": 1e-5,
           #  "fitting": True,
           #  "friction": base.get("friction", 0.0)
        





In [ ]:

# Path to your logs folder
logs_dir = "./logs"  # or whatever path you're saving them in
zip_name = "experiment_logs.zip"

# Make a ZIP file
shutil.make_archive(base_name=zip_name.replace(".zip", ""), format='zip', root_dir=logs_dir)

print(f"✅ Logs zipped successfully: {zip_name}")

In [ ]:


# Define base configurations
base_experiments = [
    {"name": "strain_sweep_0.01", "dt": 0.002, "dyn_dt_change": 0.02, "bend": 20, "strain-limit-eps": 0.01, "contact-ghat": 0.4e-3},
    {"name": "strain_sweep_0.05", "dt": 0.002, "dyn_dt_change": 0.02, "bend": 20, "strain-limit-eps": 0.05, "contact-ghat": 0.4e-3},
    {"name": "strain_sweep_0.10", "dt": 0.002, "dyn_dt_change": 0.02, "bend": 20, "strain-limit-eps": 0.1,  "contact-ghat": 0.4e-3},
    {"name": "friction_high",       "dt": 0.005, "dyn_dt_change": 0.02, "bend": 25, "strain-limit-eps": 0.02, "contact-ghat": 0.8e-3, "friction": 0.8},
    {"name": "dt_large",            "dt": 0.05,  "dyn_dt_change": 0.05, "bend": 20, "strain-limit-eps": 0.01, "contact-ghat": 1e-4},
    {"name": "contact_sweep",       "dt": 0.002, "dyn_dt_change": 0.02, "bend": 20, "strain-limit-eps": 0.02, "contact-ghat": 1e-4},
]

# Shell models to compare
shell_models = ["arap", "baraffwitkin"]

# Output base folder
run_id = datetime.now().strftime("run_%Y%m%d_%H%M%S")
log_base = f"batch_logs_{run_id}"
os.makedirs(log_base, exist_ok=True)

# Function stub (replace this with your actual run logic)
def run_experiment(params, folder):
    os.makedirs(folder, exist_ok=True)
    with open(os.path.join(folder, "params.json"), "w") as f:
        json.dump(params, f, indent=2)
    print(f"→ Launched {folder}")

# Launch all experiments
for base in base_experiments:
    for shell in shell_models:
        params = {
            "dt": base["dt"],
            "dyn_dt_change": base["dyn_dt_change"],
            "bend": base["bend"],
            "strain-limit-eps": base["strain-limit-eps"],
            "contact-ghat": base["contact-ghat"],
            "model-shell": shell,
            "frames": 240,
            "cg-max-iter": 100000,
            "cg-tol": 1e-5,
            "fitting": True,
            "friction": base.get("friction", 0.0)
        }

        folder_name = f"{base['name']}_{shell}"
        out_folder = os.path.join(log_base, folder_name)
        run_experiment(params, out_folder)


In [ ]:
# regular, single-run params

def run_simulation(shell_model, label):
    param = (
        app.session.param()
        .set("fitting")
        .set("model-shell", shell_model)
        .set("dt", 0.05)
        .set("frames", 240)
        .set("friction", 0.8)
        .set("contact-ghat", 1e-4)
        .set("strain-limit-eps", 0.01)
        .set("output-folder", f"output_{label}")
    )

    scene = app.load("fitting", param)
    session = app.session.create(scene)
    session.run()
    session.report()
    return session


param = (
    app.session.param()
    .set("fitting")
    .set("dt", 0.05)
    .set("frames", 240)
    .set("friction", 0.0)
    .set("model-shell", "arap")
    .set("contact-ghat", 1e-4)
    .set("strain-limit-eps", 0.01)
    #.set("contact-ghat", 0.4e-3) # 0.5e3  parameter controls the activation threshold of contact. Lower values increase contact density:
    #.set("strain-limit-eps", 0.06) #0.1 Make it smaller to strongly limit how much the cloth can stretch
    .set("cg-tol", 1e-5)
    .set("cg-max-iter",100000)
   # .set("bend",17)
)
param.dyn("fitting").time(0.15).hold().change(False)
param.dyn("dt").time(0.15).hold().change(0.0338)   #since 0.032 always converges but 0.034 fails 

session = app.session.create(fixed)
session.start(param).preview()
session.stream()

# Run both simulations
session_arap = run_simulation("arap", "arap")
session_bw = run_simulation("baraffwitkin", "bw")



In [153]:

# # def run_experiment(params, exp_name=None):
# def run_experiment(params, param_dict, exp_name=None):
#     build_scene()
    

#     # Build the scene (you can keep your asset loading here if static)

#     session = app.session.create(fixed)
#     session.start(params).preview()
#     session.stream()

#     while not session.finished():
#         pass
#     if session.finished():
#         print("session finished")
    

#     # Auto-name everything
#     timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
#     exp_name = exp_name or f"exp_{timestamp}"
#     exp_dir = f"./logs/{exp_name}"
#     os.makedirs(exp_dir, exist_ok=True)

#     # Save structured logs
#     logs = session.get.log.names()

#     for logname in logs:
#         numbers = session.get.log.numbers(logname)
#         with open(f"{exp_dir}/{logname}.csv", "w") as f:
#             f.write("frame,value\n")
#             for frame, value in numbers:
#                 f.write(f"{frame},{value}\n")

#     # Save stdout separately
#     lines_to_write = session.get.log.stdout()
#     with open(f"{exp_dir}/stdout.txt", "w") as f:
#         f.writelines(line + '\n' for line in lines_to_write)

#     # Save parameters used for this experiment
#     with open(f"{exp_dir}/params.txt", "w") as f:
#         for key, value in param_dict.items():
#             f.write(f"{key}: {value}\n")
#     print(f"Experiment {exp_name} saved to {exp_dir}")
    
#     return session.finished()




    


In [ ]:
experiments = [
     # {"dt": 1e-3, "dyn_dt_change": 0.01, "bend": 1}, base params
    # {"dt": 0.01, "dyn_dt_change": 0.02, "bend": 5},
    # {"dt": 0.01, "dyn_dt_change": 0.025, "bend": 10},
    # {"dt": 0.01, "dyn_dt_change": 0.027, "bend": 15},
    # {"dt": 0.01, "dyn_dt_change": 0.028, "bend": 15},
    # {"dt": 0.01, "dyn_dt_change": 0.0284, "bend": 16},
    # {"dt": 0.01, "dyn_dt_change": 0.0288, "bend": 16},
    # {"dt": 0.01, "dyn_dt_change": 0.0298, "bend": 18},
    #{"dt": 0.01, "dyn_dt_change": 0.032, "bend": 20},
    #{"dt": 0.01, "dyn_dt_change": 0.035, "bend": 20},
    {"dt": 0.01, "dyn_dt_change": 0.038, "bend": 54},
    {"dt": 0.01, "dyn_dt_change": 0.040, "bend": 56},
]
# experiments = [
#     {"dt": 0.01, "dyn_dt_change": 0.02, "bend": 5, "strain-limit-eps": 0.05},
#     {"dt": 0.01, "dyn_dt_change": 0.025, "bend": 10, "strain-limit-eps": 0.02},
#     {"dt": 0.015, "dyn_dt_change": 0.02, "bend": 5, "strain-limit-eps": 0.01},    #default strain
#     {"dt": 0.02, "dyn_dt_change": 0.025, "bend": 5, "strain-limit-eps": 0.01},    #default strain
#     {"dt": 0.01, "dyn_dt_change": 0.02, "bend": 5, "contact-ghat": 0.4e-3},
#     {"dt": 0.01, "dyn_dt_change": 0.025, "bend": 10, "contact-ghat": 0.2e-3},
# ]


# param = (
#     app.session.param()
#     .set("fitting")
#     .set("dt", 1e-3)
#     .set("frames", 240)
#     .set("friction", 0.0)
#     .set("model-shell", "arap")
#     .set("contact-ghat", 0.5e-3)
#     .set("strain-limit-eps", 0.1)
# )
# param.dyn("fitting").time(0.15).hold().change(False)
# param.dyn("dt").time(0.15).hold().change(0.01)


for idx, exp in enumerate(experiments):
    param = (
        app.session.param()
        .set("fitting")
        .set("dt", exp["dt"])
        .set("frames", 240)
        .set("friction", 0.0)
        .set("model-shell", "arap")
        .set("contact-ghat", 0.5e-3)
        .set("strain-limit-eps", 0.1)
        .set("cg-max-iter",100000)
        .set("cg-tol",0.000001)
        .set("bend", exp["bend"])
    )
    param.dyn("fitting").time(0.15).hold().change(False)
    param.dyn("dt").time(0.15).hold().change(exp["dyn_dt_change"])
    new_idx = idx+9

    sess = run_experiment(param, exp, exp_name=f"dyn_dt_change_testlogz{new_idx+1}")
    print(sess)





build: 100%|██████████████████████████| 10/10 [00:00<00:00, 42.12it/s]


#vert,#tri,#pin,#stitch_ind
"28,467","54,928","12,811",851


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(position=(0.0, 0.0, 1.8741250038146973), quaterni…

interactive(children=(FloatSlider(value=0.0, description='time', max=3.5, step=0.01), Output()), _dom_classes=…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(position=(0.0, 0.0, 1.8741250038146973), quaterni…

Button(description='Terminate Solver', style=ButtonStyle())

HTML(value='')

HTML(value='')

In [ ]:
experiments = [
    # {"dt": 0.01, "dyn_dt_change": 0.02, "bend": 5, "strain-limit-eps": 0.05,"contact-ghat": 0.5e-3},
    # {"dt": 0.01, "dyn_dt_change": 0.025, "bend": 10, "strain-limit-eps": 0.02,"contact-ghat": 0.5e-3},  does not run past 1st frame
    {"dt": 0.01, "dyn_dt_change": 0.02, "bend": 5, "strain-limit-eps": 0.01,"contact-ghat": 0.5e-3},    #default strain
    {"dt": 0.02, "dyn_dt_change": 0.025, "bend": 5, "strain-limit-eps": 0.01,"contact-ghat": 0.5e-3},    #default strain
    {"dt": 0.01, "dyn_dt_change": 0.02, "bend": 5,"strain-limit-eps": 0.01, "contact-ghat": 0.4e-3},
    {"dt": 0.01, "dyn_dt_change": 0.025, "bend": 10,"strain-limit-eps": 0.01, "contact-ghat": 0.2e-3},
]

for idx, exp in enumerate(experiments):
    param = (
        app.session.param()
        .set("fitting")
        .set("dt", exp["dt"])
        .set("frames", 240)
        .set("friction", 0.0)
        .set("model-shell", "arap")
        .set("contact-ghat", exp["contact-ghat"])
        .set("strain-limit-eps", exp["strain-limit-eps"])
        .set("cg-max-iter",100000)
        .set("cg-tol",0.000001)
        .set("bend", exp["bend"])
    )
    param.dyn("fitting").time(0.15).hold().change(False)
    param.dyn("dt").time(0.15).hold().change(exp["dyn_dt_change"])

    sess = run_experiment(param, exp, exp_name=f"strain3_run_{idx+1}")
    print(sess)

In [8]:
# param = (
#     app.session.param()
#     .set("fitting")
#     .set("dt", 0.01)
#     .set("frames", 200)
#     .set("friction", 0.0)
#     .set("model-shell", "arap")
#     .set("contact-ghat", 0.4e-3)
#     .set("strain-limit-eps", 0.1)
#     .set("bend", 5)
#     # .set("area-density", 1e3).set("volume-density", 1e5)
#     .set("cg-max-iter",1000000)
#     .set("cg-tol",0.000001)
# )
param = (
    app.session.param()
    .set("fitting")
    .set("dt", 0.015)
    .set("frames", 240)
    .set("friction", 0.0)
    .set("model-shell", "arap")
    .set("contact-ghat", 0.5e-3)
    .set("strain-limit-eps", 0.1)
    .set("cg-max-iter",100000)
    .set("cg-tol",0.000001)  #o.oo1 is orig, 0.000001 is the val from opencloth
    .set("bend", 10)
)
param.dyn("fitting").time(0.15).hold().change(False)
param.dyn("dt").time(0.15).hold().change(0.03)
session = app.session.create(fixed)
session.start(param).preview()
session.stream()

# with these params ^^, i think the dress moves more realistically, but it fails to converge at frame 128-130

# start with dt=(0.001,0.02 ) and see if it fails to converge at 128
#if it converges, increase second dt val until it fails to converge agfain 

# the mora accurate the simulation (cg-tol), the smaller the time steps we need? 
# so if cg-tol = 0.00001, then we need dt to be less than 0.001?



#param.dyn("fitting").time(0.15).hold().change(False)
#param.dyn("dt").time(0.15).hold().change(0.04)
# param.dyn("cg-max-iter").time(0.15).hold().change(10e5)
# param.dyn("cg-tol").change(0.000001)





Renderer(camera=PerspectiveCamera(children=(DirectionalLight(position=(0.0, 0.0, 1.8741250038146973), quaterni…

Button(description='Terminate Solver', style=ButtonStyle())

HTML(value='')

HTML(value='')

In [22]:
# print(session.finished())
if session.finished():
    print("finished")
else:
    print("not")
logs = session.get.log.names()




# Save stdout separately FIRST (always exists)
# lines_to_write = session.get.log.stdout()
# with open(f"{exp_dir}/stdout.txt", "w") as f:
#         f.writelines(line + '\n' for line in lines_to_write)




thread 'main' panicked at src/backend.rs:235:17:
failed to advance
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
not


In [11]:
# get a list of log names
logs = session.get.log.names()
assert "time-per-frame" in logs
assert "newton-steps" in logs
assert "pcg-iter" in logs

# get a list of time per video frame
msec_per_video = session.get.log.numbers("time-per-frame")

# compute the average time per video frame
print("avg per frame:", sum([n for _, n in msec_per_video]) / len(msec_per_video))

# get a list of newton steps
newton_steps = session.get.log.numbers("newton-steps")
pcg_iterations = session.get.log.numbers("pcg-iter")

# compute the average of consumed newton steps
print("avg newton steps:", sum([n for _, n in newton_steps]) / len(newton_steps))
print("avg pcg iterations per newton step :", sum([n for _, n in pcg_iterations]) / len(pcg_iterations))

def write_lines_to_file(filename, lines):
    """Writes each line from a list to a file.

    Args:
        filename: The name of the file to write to.
        lines: A list of strings, where each string is a line to write.
    """
    with open(filename, 'w') as file:
        file.writelines(line + '\n' for line in lines)

# Example usage:
lines_to_write = session.get.log.stdout()
write_lines_to_file("stdout_test.txt", lines_to_write)

avg per frame: 458.7063492063492
avg newton steps: 2.730769230769231
avg pcg iterations per newton step : 997.2277353689567


In [ ]:
RUST_BACKTRACE=1

In [ ]:
session.animate()

In [ ]:
session.export.animation()

In [ ]:
# this is for CI
assert session.finished()

In [ ]:
def write_lines_to_file(filename, lines):
    """Writes each line from a list to a file.

    Args:
        filename: The name of the file to write to.
        lines: A list of strings, where each string is a line to write.
    """
    with open(filename, 'w') as file:
        file.writelines(line + '\n' for line in lines)

# Example usage:
lines_to_write = session.get.log.stdout()
write_lines_to_file("ppf_sim_data_3mil.txt", lines_to_write)

# # Verify the content of the file (optional):
# with open("ppf_sim_data2.txt", 'r') as file:
#     print(file.read())


In [2]:

# Path to your logs folder
logs_dir = "./logs"  # or whatever path you're saving them in
zip_name = "experiment_logs.zip"

# Make a ZIP file
shutil.make_archive(base_name=zip_name.replace(".zip", ""), format='zip', root_dir=logs_dir)

print(f"✅ Logs zipped successfully: {zip_name}")


✅ Logs zipped successfully: experiment_logs.zip
